In [16]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
links = ['https://www2.hm.com/en_us/productpage.0427159001.html','https://www2.hm.com/en_us/productpage.0427159002.html','https://www2.hm.com/en_us/productpage.0427159003.html','https://www2.hm.com/en_us/productpage.0427159004.html','https://www2.hm.com/en_us/productpage.0427159005.html']

In [18]:
### Instantiating a Web Driver
# starting drive

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [17]:
df_comp = pd.DataFrame()

for idx, link in enumerate(links):

    # sku
    sku = link.split('.')[3]
    print(sku)

0427159001
0427159002
0427159003
0427159004
0427159005


In [19]:
df_comp = pd.DataFrame()

for idx, link in enumerate(links):

    # sku
    sku = link.split('.')[3]
    print('scraping page {}/{}: {}'.format( idx+1, len(links), link))
    
    # load web page
    driver.get(link)
    
    # get price
    # try this class (for no promo days)
    class_price = "ProductPrice-module--productItemPrice__2i2Hc"
    element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )
    price = element.text

    # if element returns empty, try this other class
    if element.text == '':
        class_price = "price.parbase"
        element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )
        price = element.text
        
        if price == '':
            price = 'NA'
    
    # get product description   
    class_desc = "ProductDescription-module--descriptionText__1zy9P"      
    # test if description exists
    try: 
        content = WebDriverWait(driver, 10).until(EC.presence_of_element_located( (By.CLASS_NAME, class_desc) ))
        desc = content.text
    except:
        desc = 'NA'
    
    # get text
    class_text = 'ProductAttributesList-module--descriptionListItem__3vUL2'
    contents = WebDriverWait(driver, timeout=10).until( EC.presence_of_all_elements_located( (By.CLASS_NAME, class_text) ) )
    
    # concatenate all lines of text
    text = str()
    # list with all text
    text = [text + line.text  for line  in contents]

    # separate fit and composition from text
    # if fit or composition is not informed they'll return NA
    fit = 'NA'
    composition = 'NA'
    for element in text:
        if 'fit' in element:
            fit = element
        if 'Composition' in element:
            composition = element    
    
    # saving raw text
    text_raw =' /'.join(text)
    
    # saving results
    df_aux = pd.DataFrame( {'sku' : sku, 'price' : price, 'fit' : fit, 'composition' : composition, 'description' : desc ,'text' : text_raw,}, index = [0] )
    df_comp = pd.concat( [df_comp, df_aux], axis = 0 )     

df_comp.reset_index(inplace = True, drop = True)

scraping page 1/5: https://www2.hm.com/en_us/productpage.0427159001.html
scraping page 2/5: https://www2.hm.com/en_us/productpage.0427159002.html
scraping page 3/5: https://www2.hm.com/en_us/productpage.0427159003.html
scraping page 4/5: https://www2.hm.com/en_us/productpage.0427159004.html
scraping page 5/5: https://www2.hm.com/en_us/productpage.0427159005.html


In [20]:
df_comp

,sku,price,fit,composition,description,text
0,0427159001,$39.99,FitSkinny fit,"CompositionCotton 91%, Polyester 7%, Spandex 2%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste..."
1,0427159002,$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste..."
2,0427159003,$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste..."
3,0427159004,$39.99,FitSkinny fit,"CompositionCotton 99%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ..."
4,0427159005,$39.99,FitSkinny fit,"CompositionCotton 72%, Polyester 20%, Modal 7%...",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste..."
